In [ ]:
API_KEY = "sk-or-v1-your-api-key"  # Replace with your actual API key

In [ ]:
#@title Imports and setup
import requests
import json
import argparse
import os
import random

API_BASE = "https://openrouter.ai/api/v1"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

In [ ]:
#@title Functions to interact with OpenRouter API
def get_free_models():
    resp = requests.get(f"{API_BASE}/models", headers=HEADERS)
    resp.raise_for_status()
    return [
        m["id"] for m in resp.json().get("data", [])
        if m.get("pricing", {}).get("request") == "0" and m.get("pricing", {}).get("prompt") == "0"
    ]

def choose_model(preferred_model=None, random=False):
    free = get_free_models()
    if not free:
        raise ValueError("No free models available.")
    if preferred_model and preferred_model in free:
        return preferred_model
    if random:  # return a random model each time
        return random.choice(free)
    return free[0] # return first free model by default

def send_to_one_model(prompt,  model_id=None, random=False, one_off=True):
    model = choose_model(model_id, random)
    if not one_off: # chat completions, several messages in an array (more expensive)
      endpoint = f"{API_BASE}/chat/completions"
      messages = [{"role": "user", "content": prompt}]
      payload = {"model": model, "messages": messages, "temperature": 0.7}
    else: # one off chat message
      endpoint = f"{API_BASE}/completions"
      payload = {"model": model, "prompt": prompt, "temperature": 0.7}
    
    resp = requests.post(endpoint, headers=HEADERS, json=payload)
    resp.raise_for_status()
    return resp.json()

def get_user_input():
    prompt = input("Enter prompt")
    if not prompt.strip():
        raise ValueError("Prompt cannot be empty.")
    
    # TODO: add optional media support
    return prompt

#@title Send your query to one model
def send_to_one():
    prompt  = get_user_input()
    model_id = input("Enter model id, r for random, or blank for default")
    if not model_id:
        model_id = None
        
    rando = False

    if model_id == "random" or model_id == "r":
        rando = True
   
    res = send_to_one_model(prompt, model_id, random=rando)
    print(json.dumps(res, indent=2))

send_to_one()

In [ ]:
#@title Media models
def get_free_media_models_tuples():
    """
    Returns a list of tuples for all free models that support at least one media input.
    Each tuple contains the model ID and a list of its supported media types.
    """
    # This set defines the media-related modalities you are interested in.
    media_types = {"image", "audio", "file"} # video is not an option in their API at the moment 
    free_media_models = []

    try:
        resp = requests.get(f"{API_BASE}/models", headers=HEADERS)
        resp.raise_for_status()
        models = resp.json().get("data", [])

        for m in models:
            pricing = m.get("pricing", {})
            # Check if the model is free
            if pricing.get("request") == "0" and pricing.get("prompt") == "0":
                modalities = set(m.get("architecture", {}).get("input_modalities", []))
                
                # Find the intersection of the model's modalities and your desired media types
                supported_media = modalities.intersection(media_types)
                
                # If the model supports at least one of the specified media types, add it to the list
                if supported_media:
                    # Append a tuple of (model_id, list_of_media)
                    free_media_models.append((m["id"], sorted(list(supported_media)))) # Sorting for consistent output

    except requests.exceptions.RequestException as e:
        print(f"An API error occurred: {e}")
        return [] # Return an empty list on error
        
    return free_media_models

# example usage
print("Free media models with supported media types:")
all_free_media = get_free_media_models_tuples() # includes the media types
print(all_free_media)

print("Free media models with supported media types (only model names):")
model_names = [x[0] for x in all_free_media] # if we only want the names of the models 
print(model_names)

In [ ]:
#@title Get models by price
def get_models_by_price(max_request_price: float, max_prompt_price: float):
    """
    Gets models with request and prompt prices less than or equal to the specified values.
    Args:
        max_request_price (float): The maximum acceptable price for a (web) request.
        max_prompt_price (float): The maximum acceptable price for a prompt/token.
    Returns:
        A list of model IDs that meet the price criteria.
    """
    affordable_models = []
    
    try:
        resp = requests.get(f"{API_BASE}/models", headers=HEADERS)
        resp.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
        models = resp.json().get("data", [])
        for m in models:
            pricing = m.get("pricing", {})
            request_price_str = pricing.get("request")
            prompt_price_str = pricing.get("prompt")
            # Ensure both pricing fields exist before trying to convert and compare
            if request_price_str is not None and prompt_price_str is not None:
                try:
                    # Convert price strings to float for numerical comparison
                    request_price = float(request_price_str)
                    prompt_price = float(prompt_price_str)
                    # Check if the model's prices are within the set budget
                    if request_price <= max_request_price and prompt_price <= max_prompt_price:
                        affordable_models.append(m["id"])
                except (ValueError, TypeError):
                    # This will catch cases where price is not a valid number (e.g., an empty string)
                    # We can simply ignore these models and continue to the next one.
                    continue
    except requests.exceptions.RequestException as e:
        print(f"An API error occurred: {e}")
        return [] # Return an empty list on error
    return affordable_models

# example usage
request_price = 0.0  # Set your maximum request price here
prompt_price = 0.0  # Set your maximum prompt price here

print("affordable models")
models = get_models_by_price(request_price, prompt_price) # set / adjust your budget here, right now its 0 for free models 
print(f"# of models: {len(models)}")
print(models)

In [ ]:
#@title WORK IN PROGRESS ADDITIONS

# WIP - media upload via google colab
def upload_media():
  # media upload
  from google.colab import files
  uploaded = files.upload()

  if not uploaded:
    return None

  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  return list(uploaded.keys())

# WIP - working but not in parrallel, media isn't supported, and doesn't handle 429 in json responses (rate limiting)
def send_to_all_free_models(prompt):
    free_models = get_free_models()
    if not free_models:
        raise ValueError("No free models available.")

    results = {}
    for model in free_models:
        try:
            print(f"Sending to: {model}")
            res = send_to_one_model(model_id=model, prompt=prompt)
            results[model] = res
        except Exception as e:
            results[model] = {"error": str(e)}
    return results

def send_to_all():
    res = send_to_all_free_models(get_user_input())
    print(json.dumps(res, indent=2))